# Hexagonal Tiles

--- Day 24: Lobby Layout ---
Your raft makes it to the tropical island; it turns out that the small crab was an excellent navigator. You make your way to the resort.

As you enter the lobby, you discover a small problem: the floor is being renovated. You can't even reach the check-in desk until they've finished installing the new tile floor.

The tiles are all hexagonal; they need to be arranged in a hex grid with a very specific color pattern. Not in the mood to wait, you offer to help figure out the pattern.

The tiles are all white on one side and black on the other. They start with the white side facing up. The lobby is large enough to fit whatever pattern might need to appear there.

A member of the renovation crew gives you a list of the tiles that need to be flipped over (your puzzle input). Each line in the list identifies a single tile that needs to be flipped by giving a series of steps starting from a reference tile in the very center of the room. (Every line starts from the same reference tile.)

Because the tiles are hexagonal, every tile has six neighbors: `east, southeast, southwest, west, northwest, and northeast`. These directions are given in your list, respectively, as `e, se, sw, w, nw, and ne`. A tile is identified by a series of these directions with no delimiters; for example, esenee identifies the tile you land on if you start at the reference tile and then move one tile east, one tile southeast, one tile northeast, and one tile east.

Each time a tile is identified, it flips from white to black or from black to white. Tiles might be flipped more than once. For example, a line like `esew` flips a tile immediately adjacent to the reference tile, and a line like `nwwswee` flips the reference tile itself.

Here is a larger example:
```
sesenwnenenewseeswwswswwnenewsewsw
neeenesenwnwwswnenewnwwsewnenwseswesw
seswneswswsenwwnwse
nwnwneseeswswnenewneswwnewseswneseene
swweswneswnenwsewnwneneseenw
eesenwseswswnenwswnwnwsewwnwsene
sewnenenenesenwsewnenwwwse
wenwwweseeeweswwwnwwe
wsweesenenewnwwnwsenewsenwwsesesenwne
neeswseenwwswnwswswnw
nenwswwsewswnenenewsenwsenwnesesenew
enewnwewneswsewnwswenweswnenwsenwsw
sweneswneswneneenwnewenewwneswswnese
swwesenesewenwneswnwwneseswwne
enesenwswwswneneswsenwnewswseenwsese
wnwnesenesenenwwnenwsewesewsesesew
nenewswnwewswnenesenwnesewesw
eneswnwswnwsenenwnwnwwseeswneewsenese
neswnwewnwnwseenwseesewsenwsweewe
wseweeenwnesenwwwswnew
```
In the above example, 10 tiles are flipped once (to black), and 5 more are flipped twice (to black, then back to white). After all of these instructions have been followed, a total of 10 tiles are black.

Go through the renovation crew's list and determine which tiles they need to flip. After all of the instructions have been followed, how many tiles are left with the black side up?

Your puzzle answer was 450.

## Second part

The tile floor in the lobby is meant to be a living art exhibit. Every day, the tiles are all flipped according to the following rules:

Any black tile with zero or more than 2 black tiles immediately adjacent to it is flipped to white.
Any white tile with exactly 2 black tiles immediately adjacent to it is flipped to black.
Here, tiles immediately adjacent means the six tiles directly touching the tile in question.

The rules are applied simultaneously to every tile; put another way, it is first determined which tiles need to be flipped, then they are all flipped at the same time.

In the above example, the number of black tiles that are facing up after the given number of days has passed is as follows:
```
Day 1: 15
Day 2: 12
Day 3: 25
Day 4: 14
Day 5: 23
Day 6: 28
Day 7: 41
Day 8: 37
Day 9: 49
Day 10: 37

Day 20: 132
Day 30: 259
Day 40: 406
Day 50: 566
Day 60: 788
Day 70: 1106
Day 80: 1373
Day 90: 1844
Day 100: 2208
```
After executing this process a total of 100 times, there would be 2208 black tiles facing up.

How many tiles will be black after 100 days?

Your puzzle answer was 4059.

In [1]:
import os
import time
import numpy as np
from collections import Counter, defaultdict
#import itertools
import re
#from collections import deque

In [2]:
DAY = 'Day_24'
FILE_END = '_01.txt'
RUN_WITH = 'input' ## admitted values: sample input 
SAMPLE_DATA = '''sesenwnenenewseeswwswswwnenewsewsw
neeenesenwnwwswnenewnwwsewnenwseswesw
seswneswswsenwwnwse
nwnwneseeswswnenewneswwnewseswneseene
swweswneswnenwsewnwneneseenw
eesenwseswswnenwswnwnwsewwnwsene
sewnenenenesenwsewnenwwwse
wenwwweseeeweswwwnwwe
wsweesenenewnwwnwsenewsenwwsesesenwne
neeswseenwwswnwswswnw
nenwswwsewswnenenewsenwsenwnesesenew
enewnwewneswsewnwswenweswnenwsenwsw
sweneswneswneneenwnewenewwneswswnese
swwesenesewenwneswnwwneseswwne
enesenwswwswneneswsenwnewswseenwsese
wnwnesenesenenwwnenwsewesewsesesew
nenewswnwewswnenesenwnesewesw
eneswnwswnwsenenwnwnwwseeswneewsenese
neswnwewnwnwseenwseesewsenwsweewe
wseweeenwnesenwwwswnew'''

In [3]:
start = time.time()

In [4]:
input_path = os.path.join(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))), "Inputs")
input_path_day = os.path.join(input_path, DAY)
file_path = os.path.join(input_path_day, DAY+ FILE_END)

### Read input

## Part I

In [5]:
# my coordinates navigator (originally started with 1 & sqrt(3) but wasn't adding anything and made it more difficult to understand)
mapping = {
    'e':(2,0),
    'se':(1,-1),
    'sw':(-1,-1),
    'w':(-2,0),
    'nw':(-1,1),
    'ne':(1,1),
    'self':(0,0)
        }
        

In [6]:
def get_values(RUN_WITH:str):
    if RUN_WITH == 'input':
        with open(file_path,'r') as f:
            values = f.read()
    elif RUN_WITH == 'sample':
        values =  SAMPLE_DATA

    else:
        print("No valid input selected")
    values = values.splitlines()
    try:
        values.remove("")
    except:
        pass
    return values

def default():
    return 0

def get_tiles(values, pattern, mapping):
    tiles = defaultdict(default)
    search = re.compile(pattern)
    for val in values:
        instructions = Counter(search.findall(val))
        pos = [0,0]
        for key, val in instructions.items():
            pos[0] += mapping[key][0] * val
            pos[1] += mapping[key][1] * val
        tiles[tuple(pos)] +=1
    return tiles

def get_black_tiles(tiles):
    black = {}
    for k,v in tiles.items():
        if v%2 != 0:
            black[k] = 1
    return black

def part_1(source):
    black_tiles = get_black_tiles(get_tiles(get_values(source),'e|se|sw|w|nw|ne', mapping ))
    return len(black_tiles), black_tiles

In [7]:
answer, black_tiles = part_1('input')
answer

450

In [8]:
%%timeit
part_1('input')

9.97 ms ± 1.16 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
## Part II

def play_day(black_tiles):
    # create the next board: all the black tiles and all the adjacent tiles to the black tiles
    next_board = defaultdict(default)
    for key in black_tiles.keys():
        for k,v in mapping.items():
            try:
                next_board[tuple((key[0]+v[0], key[1] +v[1]))] =[0,0]
            except:
                print(key, k, v)
                return None

    # set black:
    for key in black_tiles.keys():
        next_board[key] = [1,0]
        
    next_black = {}
    #lookup
    for tile, vals in next_board.items():
        for key in mapping.keys():
            if key != 'self':
                lookup = tuple((tile[0] + mapping[key][0], tile[1] + mapping[key][1]))
                if lookup in black_tiles.keys():
                    next_board[tile][1] +=1

        if vals[0] == 1: #black tile
            if vals[1] in (1,2):
                next_black[tile] = 1
        else: # white tile
            if vals[1] == 2:
                next_black[tile] = 1
    return next_black

In [10]:
def part_2(days, black_tiles):
    for i in range(days):
        black_tiles = play_day(black_tiles)
    return len(black_tiles), black_tiles

In [11]:
ans, black_tiles = part_2(100, black_tiles)
ans

4059